In [12]:
import re,pickle
from janome.tokenizer import Tokenizer

In [13]:
def textToWords(text):
    text = re.sub("《[^》]+》","",text)
    text = re.sub("【[^]+]】","",text)
    text = re.sub("[「」　|\n]","",text)

    seperator = "。"
    text_list = text.split(seperator)

    t = Tokenizer()
    words = []
    for sentence in text_list:
        words.append(list(t.tokenize(sentence,wakati=True)))

    with open("./data/newWords.pickle","wb") as f:
        pickle.dump(words,f)


In [14]:
with open("./data/wagahaiwa_nekodearu.txt",mode="r",encoding="utf-8") as f:
  wagahai_original = f.read()

In [15]:
textToWords(wagahai_original)

In [16]:
with open("./data/newWords.pickle","rb") as f:
    wagahai_words = pickle.load(f)
print(wagahai_words[:2])

[['吾輩', 'は', '猫', 'で', 'ある'], ['名前', 'は', 'まだ', '無い']]


In [17]:
from gensim.models import word2vec

In [18]:
model = word2vec.Word2Vec(
    wagahai_words,
    vector_size = 100,
    min_count = 5,
    window = 5,
    epochs = 20,
    sg = 0,
)

In [19]:
print(model.wv.vectors.shape)
print(model.wv.vectors)

(3351, 100)
[[-0.8729195  -0.15787932  0.66140246 ... -0.2673085   0.201834
   0.2817939 ]
 [-0.83868855  0.4674512  -0.09390387 ... -0.94457126 -1.3329856
   0.22123514]
 [-0.57254404  0.298663    0.3805712  ... -1.0402544   0.5384641
  -0.5327017 ]
 ...
 [-0.20675874  0.17718895 -0.01035172 ... -0.15477373 -0.0273036
  -0.08677949]
 [-0.13624665  0.08977344 -0.04096837 ... -0.0429566  -0.04364458
  -0.02132391]
 [-0.2568871   0.10417002 -0.02762192 ... -0.18603489  0.02581191
  -0.04042498]]


In [20]:
print(len(model.wv.index_to_key))
print(model.wv.index_to_key[:4])

3351
['の', 'て', '、', 'に']


In [21]:
print(model.wv.vectors[0])
print(model.wv.__getitem__("の"))

[-0.8729195  -0.15787932  0.66140246  0.25933507 -1.5687203   0.40362045
  0.2652638   0.16017683 -0.33082148 -0.12873554  0.24850586  0.1715102
  0.11631247 -0.20005113 -0.9538881  -0.02318425 -0.37491703 -0.330247
 -0.47138414  0.22570907  0.26643673  0.19234857  0.8122489  -0.8686416
 -0.49978486 -0.29664403  0.6111641   0.25056452 -0.01308375 -0.33243725
 -0.08966189 -0.56099975  0.6734196   0.2168586   0.43009484 -0.24378628
  0.20576906 -1.3983612   0.2816132   0.05981953 -0.1836564   0.21405669
 -0.45556304 -0.14092511 -0.07182103  0.17339164  0.05318012  0.69290644
  0.40710765  0.23072982 -0.40346247  0.24576183  0.3540368  -0.0836683
  0.25652814 -0.29778257 -0.09547439  0.2234872  -0.2900447   0.1896612
  0.2523877   1.0973657  -0.68365544 -0.9070066   0.33780646  0.14212166
  0.09793854  1.076369   -0.5242488  -0.04818152  0.51900727  1.1851277
  0.35349914 -1.0453597  -0.16083673  0.27978536  0.9602716   0.1913727
 -0.10478798  0.38777047 -0.263614   -0.06414632  0.1284309

In [22]:
print(model.wv.most_similar("猫"))

[('人間', 0.7245374321937561), ('教師', 0.6979549527168274), ('君子', 0.6854879856109619), ('充分', 0.6774835586547852), ('逆上', 0.6725793480873108), ('本人', 0.6450510025024414), ('罪', 0.6346117258071899), ('動物', 0.6286435127258301), ('世間', 0.6271816492080688), ('一般', 0.6193382143974304)]


In [24]:
import numpy as np

In [25]:
a = model.wv.__getitem__('猫')
b = model.wv.__getitem__('人間')
cos_sim = np.dot(a,b)/np.linalg.norm(a)/np.linalg.norm(b)
print(cos_sim)

0.7245375


In [26]:
model.wv.most_similar(positive=['猫','人間'])

[('逆上', 0.771516740322113),
 ('君子', 0.770756721496582),
 ('世間', 0.759636402130127),
 ('充分', 0.7345070242881775),
 ('事実', 0.7130858302116394),
 ('彼等', 0.7109562158584595),
 ('動物', 0.7026592493057251),
 ('本人', 0.6950827836990356),
 ('教師', 0.688463032245636),
 ('一般', 0.688091516494751)]

In [27]:
model.wv.most_similar(positive=['人間','猫'],negative=['夢'])

[('教師', 0.7083929181098938),
 ('芸術', 0.6811930537223816),
 ('君子', 0.6618895530700684),
 ('充分', 0.6544024348258972),
 ('実業', 0.62453293800354),
 ('家', 0.6181164979934692),
 ('出来る', 0.617882490158081),
 ('事実', 0.609535276889801),
 ('罪', 0.6042826175689697),
 ('世間', 0.6011515259742737)]

In [28]:
model.wv.most_similar(positive=['教師'],negative=['夢'])

[('えらい', 0.6548981666564941),
 ('嫌', 0.6288498044013977),
 ('しかし', 0.5983120799064636),
 ('月並', 0.5797993540763855),
 ('なかなか', 0.5762020349502563),
 ('家', 0.5730352997779846),
 ('伯父', 0.5675895810127258),
 ('いえ', 0.5655837059020996),
 ('死ぬ', 0.5648002624511719),
 ('あれ', 0.5566771626472473)]